<img src="https://raw.githubusercontent.com/dask/dask/main/docs/source/images/dask_horizontal_no_pad.svg"
     width="30%"
     alt="Dask logo\" />

# Parallel and Distributed Machine Learning

The material in this notebook was based on the open-source content from [Dask's tutorial repository](https://github.com/dask/dask-tutorial) and the [Machine learning notebook](https://github.com/coiled/data-science-at-scale/blob/master/3-machine-learning.ipynb) from data science at scale from coiled

So far we have seen how Dask makes data analysis scalable with parallelization via Dask DataFrames. Let's now see how [Dask-ML](https://ml.dask.org/) allows us to do machine learning in a parallel and distributed manner. Note, machine learning is really just a special case of data analysis (one that automates analytical model building), so the 💪 Dask gains 💪 we've seen will apply here as well!

(If you'd like a refresher on the difference between parallel and distributed computing, [here's a good discussion on StackExchange](https://cs.stackexchange.com/questions/1580/distributed-vs-parallel-computing).)


## Types of scaling problems in machine learning

There are two main types of scaling challenges you can run into in your machine learning workflow: scaling the **size of your data** and scaling the **size of your model**. That is:

1. **CPU-bound problems**: Data fits in RAM, but training takes too long. Many hyperparameter combinations, a large ensemble of many models, etc.
2. **Memory-bound problems**: Data is larger than RAM, and sampling isn't an option.

Here's a handy diagram for visualizing these problems:

<img src="https://raw.githubusercontent.com/coiled/data-science-at-scale/master/images/dimensions_of_scale.svg"
     width="60%"
     alt="scaling problems\" />


In the bottom-left quadrant, your datasets are not too large (they fit comfortably in RAM) and your model is not too large either. When these conditions are met, you are much better off using something like scikit-learn, XGBoost, and similar libraries. You don't need to leverage multiple machines in a distributed manner with a library like Dask-ML. However, if you are in any of the other quadrants, distributed machine learning is the way to go.

Summarizing: 

* For in-memory problems, just use scikit-learn (or your favorite ML library).
* For large models, use `dask_ml.joblib` and your favorite scikit-learn estimator.
* For large datasets, use `dask_ml` estimators.

## Scikit-Learn in five minutes

<img src="https://raw.githubusercontent.com/coiled/data-science-at-scale/master/images/scikit_learn_logo_small.svg" 
     width="30%"
     alt="sklearn logo\" />


In this section, we'll quickly run through a typical Scikit-Learn workflow:

* Load some data (in this case, we'll generate it)
* Import the Scikit-Learn module for our chosen ML algorithm
* Create an estimator for that algorithm and fit it with our data
* Inspect the learned attributes
* Check the accuracy of our model

Scikit-Learn has a nice, consistent API:

* You instantiate an `Estimator` (e.g. `LinearRegression`, `RandomForestClassifier`, etc.). All of the models *hyperparameters* (user-specified parameters, not the ones learned by the estimator) are passed to the estimator when it's created.
* You call `estimator.fit(X, y)` to train the estimator.
* Use `estimator` to inspect attributes, make predictions, etc. 

Here `X` is an array of *feature variables* (what you're using to predict) and `y` is an array of *target variables* (what we're trying to predict).

### Generate some random data

In [9]:
from sklearn.datasets import make_classification

# Generate data
X, y = make_classification(n_samples=10000, n_features=4, random_state=0)

**Refreshing some ML concepts**

- `X` is the samples matrix (or design matrix). The size of `X` is typically (`n_samples`, `n_features`), which means that samples are represented as rows and features are represented as columns.
- A "feature" (also called an "attribute") is a measurable property of the phenomenon we're trying to analyze. A feature for a dataset of employees might be their hire date, for example.
- `y` are the target values, which are real numbers for regression tasks, or integers for classification (or any other discrete set of values). For unsupervized learning tasks, `y` does not need to be specified. `y` is usually 1d array where the `i`th entry corresponds to the target of the `i`th sample (row) of `X`.

In [10]:
# Let's take a look at X
X[:8]

array([[-0.77244139,  0.3607576 , -2.38110133,  0.08757   ],
       [ 1.14946035,  0.62254594,  0.37302939,  0.45965795],
       [-1.90879217, -1.1602627 , -0.27364545, -0.82766028],
       [-0.77694695,  0.31434299, -2.26231851,  0.06339125],
       [-1.17047054,  0.02212382, -2.17376797, -0.13421976],
       [ 0.79010037,  0.68530624, -0.44740487,  0.44692959],
       [ 1.68616989,  1.6329131 , -1.42072654,  1.04050557],
       [-0.93912893, -1.02270838,  1.10093827, -0.63714432]])

In [11]:
# Let's take a look at y
y[:8]

array([0, 0, 1, 0, 0, 0, 0, 1])

### Fitting and SVC

For this example, we will fit a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

In [12]:
from sklearn.svm import SVC

estimator = SVC(random_state=0)
estimator.fit(X, y)

SVC(random_state=0)

We can inspect the learned features by taking a look a the `support_vectors_`:

In [13]:
estimator.support_vectors_[:4]

array([[-0.77244139,  0.3607576 , -2.38110133,  0.08757   ],
       [ 1.14946035,  0.62254594,  0.37302939,  0.45965795],
       [-0.77694695,  0.31434299, -2.26231851,  0.06339125],
       [ 0.79010037,  0.68530624, -0.44740487,  0.44692959]])

And we check the accuracy:

In [14]:
estimator.score(X, y)

0.905

There are [3 different approaches](https://scikit-learn.org/0.15/modules/model_evaluation.html) to evaluate the quality of predictions of a model. One of them is the **estimator score method**. Estimators have a score method providing a default evaluation criterion for the problem they are designed to solve, which is discussed in each estimator's documentation.

### Hyperparameter Optimization

There are a few ways to learn the best *hyper*parameters while training. One is `GridSearchCV`.
As the name implies, this does a brute-force search over a grid of hyperparameter combinations. Scikit-learn provides tools to automatically find the best parameter combinations via cross-validation (which is the "CV" in `GridSearchCV`).

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
%%time
estimator = SVC(gamma='auto', random_state=0, probability=True)
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

# Brute-force search over a grid of hyperparameter combinations
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2)
grid_search.fit(X, y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] END ................................C=0.001, kernel=rbf; total time=   3.0s
[CV] END ................................C=0.001, kernel=rbf; total time=   2.9s
[CV] END ...............................C=0.001, kernel=poly; total time=   1.2s
[CV] END ...............................C=0.001, kernel=poly; total time=   1.1s
[CV] END .................................C=10.0, kernel=rbf; total time=   0.8s
[CV] END .................................C=10.0, kernel=rbf; total time=   0.8s
[CV] END ................................C=10.0, kernel=poly; total time=   1.1s
[CV] END ................................C=10.0, kernel=poly; total time=   1.0s
CPU times: user 14 s, sys: 204 ms, total: 14.2 s
Wall time: 14.2 s


GridSearchCV(cv=2,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             param_grid={'C': [0.001, 10.0], 'kernel': ['rbf', 'poly']},
             verbose=2)

In [17]:
grid_search.best_params_, grid_search.best_score_

({'C': 10.0, 'kernel': 'rbf'}, 0.9086000000000001)

## Compute Bound: Single-machine parallelism with Joblib

<img src="https://raw.githubusercontent.com/coiled/data-science-at-scale/master/images/joblib_logo.svg" 
     alt="Joblib logo" 
     width="15%"/>

In this section we'll see how [Joblib](https://joblib.readthedocs.io/en/latest/) ("*a set of tools to provide lightweight pipelining in Python*") gives us parallelism on our laptop. Here's what our grid search graph would look like if we set up six training "jobs" in parallel:

<img src="https://raw.githubusercontent.com/coiled/data-science-at-scale/master/images/unmerged_grid_search_graph.svg" 
     alt="grid search graph" 
     width="75%"/>

With Joblib, we can say that Scikit-Learn has *single-machine* parallelism.
Any Scikit-Learn estimator that can operate in parallel exposes an `n_jobs` keyword, which tells you how many tasks to run in parallel. Specifying `n_jobs=-1` jobs means running the maximum possible number of tasks in parallel.

In [18]:
%%time
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)
grid_search.fit(X, y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
CPU times: user 2.28 s, sys: 122 ms, total: 2.4 s
Wall time: 6.82 s


GridSearchCV(cv=2,
             estimator=SVC(gamma='auto', probability=True, random_state=0),
             n_jobs=-1,
             param_grid={'C': [0.001, 10.0], 'kernel': ['rbf', 'poly']},
             verbose=2)

Notice that the computation above it is faster than before. If you are running this computation on binder, you might not see a speed-up and the reason for that is that binder instances tend to have only one core with no threads so you can't see any parallelism. 

## Compute Bound: Multi-machine parallelism with Dask


In this section we'll see how Dask (plus Joblib and Scikit-Learn) gives us multi-machine parallelism. Here's what our grid search graph would look like if we allowed Dask to schedule our training "jobs" over multiple machines in our cluster:

<img src="https://raw.githubusercontent.com/coiled/data-science-at-scale/master/images/merged_grid_search_graph.svg" 
     alt="merged grid search graph" 
     width="100%"/>
     
We can say that Dask can talk to Scikit-Learn (via Joblib) so that our *cluster* is used to train a model. 

If we run this on a laptop, it will take quite some time, but the CPU usage will be satisfyingly near 100% for the duration. To run faster, we would need a distributed cluster. For details on how to create a LocalCluster you can check the Dask documentation on [Single Machine: dask.distributed](https://docs.dask.org/en/latest/setup/single-distributed.html). 

Let's instantiate a Client with `n_workers=4`, which will give us a `LocalCluster`.

In [19]:
import dask.distributed

client = dask.distributed.Client(n_workers=4)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53865,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:53877,Total threads: 2
Dashboard: http://127.0.0.1:53881/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:53867,


**Note:** Click on Cluster Info, to see more details about the cluster. You can see the configuration of the cluster and some other specs. 

We can expand our problem by specifying more hyperparameters before training, and see how using `dask` as backend can help us. 

In [20]:
param_grid = {
    'C': [0.001, 0.1, 1.0, 2.5, 5, 10.0],
    'kernel': ['rbf', 'poly', 'linear'],
    'shrinking': [True, False],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)

### Dask parallel backend

We can fit our estimator with multi-machine parallelism by quickly *switching to a Dask parallel backend* when using joblib. 

In [21]:
import joblib

In [22]:
%%time
with joblib.parallel_backend("dask", scatter=[X, y]):
    grid_search.fit(X, y)

Fitting 2 folds for each of 36 candidates, totalling 72 fits
CPU times: user 5.03 s, sys: 457 ms, total: 5.48 s
Wall time: 27 s


distributed.utils - ERROR - '<' not supported between instances of 'NoneType' and 'NoneType'
Traceback (most recent call last):
  File "/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/utils.py", line 638, in log_errors
    yield
  File "/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/dashboard/components/scheduler.py", line 3117, in graph_doc
    graph = TaskGraph(scheduler, sizing_mode="stretch_both")
  File "/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/dashboard/components/scheduler.py", line 1947, in __init__
    self.layout = GraphLayout(scheduler)
  File "/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/diagnostics/graph_layout.py", line 38, in __init__
    self.update_graph(
  File "/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/diagnostics/graph_layout.py", line 48

[CV] END .................................C=10.0, kernel=rbf; total time=   1.3s
[CV] END .................................C=10.0, kernel=rbf; total time=   1.4s
[CV] END ................................C=10.0, kernel=poly; total time=   1.6s
[CV] END ................................C=10.0, kernel=poly; total time=   1.8s
[CV] END ...............................C=0.001, kernel=poly; total time=   1.9s
[CV] END ...............................C=0.001, kernel=poly; total time=   1.9s
[CV] END ................................C=0.001, kernel=rbf; total time=   3.7s
[CV] END ................................C=0.001, kernel=rbf; total time=   3.7s


**What just happened?**

Dask-ML developers worked with the Scikit-Learn and Joblib developers to implement a Dask parallel backend. So internally, scikit-learn now talks to Joblib, and Joblib talks to Dask, and Dask is what handles scheduling all of those tasks on multiple machines.

The best parameters and best score:

In [ ]:
grid_search.best_params_, grid_search.best_score_

## Memory Bound: Multi-machine parallelism with Dask-ML

We have seen how to work with larger models, but sometimes you'll want to train on a larger than memory dataset. `dask-ml` has implemented estimators that work well on Dask `Arrays` and `DataFrames` that may be larger than your machine's RAM.

In [23]:
from dask_ml.datasets import make_regression
from dask_ml.linear_model import LinearRegression

In [24]:
X, y = make_regression(n_samples=10_000, chunks=100)

In [25]:
X

dask.array<normal, shape=(10000, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [26]:
lr = LinearRegression()

In [27]:
%%time
lr.fit(X, y)

KeyboardInterrupt: 

In [28]:
lr.predict(X)

AttributeError: 'LinearRegression' object has no attribute '_coef'

In [ ]:
lr.score(X, y)

distributed.nanny - WARNING - Restarting worker


[CV] END .............C=0.001, kernel=linear, shrinking=True; total time=   1.6s
[CV] END ............C=0.001, kernel=linear, shrinking=False; total time=   1.2s
[CV] END ................C=0.001, kernel=rbf, shrinking=True; total time=   5.5s
[CV] END .................C=0.1, kernel=poly, shrinking=True; total time=   1.2s
[CV] END ..............C=0.1, kernel=linear, shrinking=False; total time=   1.7s
[CV] END ...............C=0.001, kernel=rbf, shrinking=False; total time=   6.4s
[CV] END ...............C=1.0, kernel=linear, shrinking=True; total time=   2.3s
[CV] END ..............C=1.0, kernel=linear, shrinking=False; total time=   2.2s
[CV] END ....................C=5, kernel=rbf, shrinking=True; total time=   1.6s
[CV] END ..............C=2.5, kernel=linear, shrinking=False; total time=   3.4s
[CV] END ...................C=5, kernel=rbf, shrinking=False; total time=   1.7s
[CV] END .................C=10.0, kernel=rbf, shrinking=True; total time=   1.7s
[CV] END ..................C

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


This works...yay!

But it does take a while...can we speed this up even more?

In [29]:
client.close()

##  Multi-machine parallelism in the cloud with Coiled

<br>
<img src="https://raw.githubusercontent.com/coiled/data-science-at-scale/master/images/Coiled-Logo_Horizontal_RGB_Black.png"
     alt="Coiled logo" 
     width=25%/>
<br>

In this section we'll see how Coiled allows us to solve machine learning problems with multi-machine parallelism in the cloud.

Coiled, [among other things](https://coiled.io/product/), provides hosted and scalable Dask clusters. The biggest barriers to entry for doing machine learning at scale are "Do you have access to a cluster?" and "Do you know how to manage it?" Coiled solves both of those problems. 

We'll spin up a Coiled cluster (with 10 workers in this case), then instantiate a Dask Client to use with that cluster.

If you are running on your local machine and not in binder, and you want to give Coiled a try, you can signup [here](https://cloud.coiled.io/login?redirect_uri=/) and you will get some free credits. If you installed the environment by following the steps on the repository's [README](https://github.com/coiled/dask-mini-tutorial/blob/main/README.md) you will have `coiled` installed. You will just need to login, by following the steps on the [setup page](https://docs.coiled.io/user_guide/getting_started.html), and you will be ready to go. 

To learn more about how to set up an environment you can visit Coiled documentation on [Creating software environments](https://docs.coiled.io/user_guide/software_environment_creation.html). But for now you can use the envioronment we set up for this tutorial. 

In [30]:
import coiled
from dask.distributed import Client

In [33]:
coiled.Cloud()

Invalid Coiled token encountered. Typically this occurs after logging out of 
https://cloud.coiled.io which resets your token for security reasons.

Please login to https://cloud.coiled.io/profile to get your token

Token:

  ········································


Authentication successful

Save credentials for next time? [Y/n]:  


Credentials have been saved at /Users/rpelgrim/.config/dask/coiled.yaml

<Cloud: rrpelgrim@https://cloud.coiled.io - running>

In [34]:
# Spin up a Coiled cluster, instantiate a Client
cluster = coiled.Cluster(
    name="dask-mini-tutorial", 
)

Output()

Using existing cluster: 'dask-mini-tutorial'

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x12a069b50>>, <Task finished name='Task-55963' coro=<Cluster._close() done, defined at /Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/coiled/context.py:53> exception=ValueError('No cluster available to close!')>)
Traceback (most recent call last):
  File "/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/tornado/ioloop.py", line 765, in _discard_future_result
    future.result()
  File "/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/coiled/context.py", line 56, in wrapper
    return await func(*args, **kwargs)
  File "/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/sit

In [35]:
client = Client(cluster)
client

/Users/rpelgrim/mambaforge/envs/dask-mini-tutorial/lib/python3.9/site-packages/distributed/client.py:1105: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| numpy   | 1.21.3 | 1.21.2    | 1.21.2  |
| pandas  | 1.3.4  | 1.3.3     | 1.3.3   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tls://10.4.2.234:8786' processes=20 threads=40, memory=151.19 GiB>

## Same Linear Regression Model as before

We can use Dask-ML estimators on the cloud to work with larger datasets.

In [36]:
X, y = make_regression(n_samples=10_000, chunks=100)

In [37]:
X

dask.array<normal, shape=(10000, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [38]:
lr = LinearRegression()

In [39]:
%%time
lr.fit(X, y)

CPU times: user 867 ms, sys: 49.7 ms, total: 917 ms
Wall time: 17.8 s


LinearRegression()

In [40]:
lr.predict(X)

dask.array<tensordot, shape=(10000,), dtype=float64, chunksize=(100,), chunktype=numpy.ndarray>

In [41]:
lr.score(X, y)

0.9999915557428719

In [ ]:
client.close()

## Extra resources:

- [Dask-ML documentation](https://ml.dask.org/)
- [Getting started with Coiled](https://docs.coiled.io/user_guide/getting_started.html)